# Técnicas de Regularización en regresión lineal

La regularización es una técnica clave en Machine Learning que nos ayuda a evitar el sobreajuste y mejorar la generalización de los modelos. Existen tres métodos principales de regularización para la regresión lineal:

- Lasso (L1)
- Ridge (L2)
- Elastic Net (L1 + L2 combinadas)

## ¿Qué es la regularización y por qué es importante?

En regresión lineal, buscamos encontrar los coeficientes
β que minimicen el error cuadrático medio (MSE):

$$\min_{\beta} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

Sin embargo, cuando tenemos muchas variables predictoras (X), el modelo puede ajustarse demasiado a los datos de entrenamiento (sobreajuste), capturando incluso ruido irrelevante.

La regularización añade una penalización a los coeficientes del modelo, forzando a que algunos sean más pequeños o incluso cero.

## Ridge Regression (Regularización L2)

En Ridge, se agrega una penalización a la suma de los cuadrados de los coeficientes ($\ell_2$):

$$
\min_{\beta} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \lambda \sum_{j=1}^{p} \beta_j^2
$$

Donde:

- λ → Controla la penalización. Si λ es alto, los coeficientes serán más pequeños.
- $\sum \beta_j^2$ → La regularización hace que los coeficientes sean pequeños, pero nunca cero.

### Ventajas de Ridge:

- Reduce el sobreajuste cuando hay multicolinealidad (variables altamente correlacionadas).
- Útil cuando todas las variables son importantes y queremos reducir su influencia sin eliminarlas.

## Lasso Regression (Regularización L1)

Lasso aplica una penalización a la suma del valor absoluto de los coeficientes ($\ell_1$):

$$
\min_{\beta} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \lambda \sum_{j=1}^{p} |\beta_j|
$$

### Diferencia clave con Ridge:

Lasso puede hacer que algunos coeficientes sean exactamente cero, eliminando variables irrelevantes.
Esto significa que también actúa como un método de selección de características.

### Ventajas de Lasso:

- Elimina variables irrelevantes, lo que simplifica el modelo.
- Útil cuando hay muchas variables y queremos seleccionar las más importantes.


## Elastic Net (combinación de L1 y L2)

Elastic Net combina Ridge y Lasso, usando una combinación de penalización L1 y L2:

$$
\min_{\beta} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \lambda_1 \sum_{j=1}^{p} |\beta_j| + \lambda_2 \sum_{j=1}^{p} \beta_j^2
$$

### Ventajas de Elastic Net:

- Elimina variables irrelevantes (como Lasso), pero mantiene estabilidad en los coeficientes (como Ridge).
- Útil cuando hay muchas variables y algunas están altamente correlacionadas.
- Permite ajustar el peso entre L1 y L2 con un hiperparámetro α:
<pre>
            α × L1 + (1−α) × L2
</pre>

- Si α=1 → Es Lasso puro.
- Si α=0 → Es Ridge puro.
- Si α está entre 0 y 1, obtenemos un equilibrio entre ambos.

## Ejemplo práctico

En este ejemplo la regresión lineal estándar no funciona bien, pero las técnicas de regularización (Ridge, Lasso, Elastic Net) mejoran los resultados. 📊

Generaremos 20 variables predictoras $(X_1, X_2, \dots, X_{20})$, pero solo 3 tienen influencia real en Y.

La regresión lineal sin regularización ajusta todos los coeficientes, incluso los de las variables irrelevantes, lo que provoca sobreajuste y un mayor error.

Ridge, Lasso y Elastic Net penalizan coeficientes innecesarios y mejoran la generalización.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

SEED = 42
TEST_SIZE = 0.2
TOTAL_SAMPLES = 200

# Generar datos con 20 variables (17 irrelevantes) y ruido
np.random.seed(SEED)
X = np.random.rand(TOTAL_SAMPLES, 20) * 10
y = 4 * X[:, 2] - 2.5 * X[:, 5] + 3 * X[:, 7] + np.random.randn(TOTAL_SAMPLES) * 10

# Dividir en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=SEED)

In [ ]:
# Ajustar modelo de regresión lineal sin regularización
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lr = lin_reg.predict(X_test)

# Evaluar rendimiento sin regularización
mse_lr = mean_squared_error(y_test, y_pred_lr)

mse_lr

128.93058897731407

In [ ]:
# Escoger los mejores hiperparámetros para Ridge, Lasso y Elastic y construir los modelos

# Definir rangos de hiperparámetros para cada modelo
param_grid_ridge = {'alpha': np.logspace(-3, 2, 10)}
param_grid_lasso = {'alpha': np.logspace(-3, 2, 10)}
param_grid_elastic = {'alpha': np.logspace(-3, 2, 10), 'l1_ratio': np.linspace(0.1, 0.9, 5)}

# GridSearchCV para encontrar mejores hiperparámetros
ridge_cv = GridSearchCV(Ridge(), param_grid_ridge, cv=5, scoring='neg_mean_squared_error')
lasso_cv = GridSearchCV(Lasso(), param_grid_lasso, cv=5, scoring='neg_mean_squared_error')
elastic_cv = GridSearchCV(ElasticNet(), param_grid_elastic, cv=5, scoring='neg_mean_squared_error')

# Entrenar modelos con los mejores hiperparámetros
ridge_cv.fit(X_train, y_train)
lasso_cv.fit(X_train, y_train)
elastic_cv.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros
best_params = {
    "Ridge": ridge_cv.best_params_,
    "Lasso": lasso_cv.best_params_,
    "Elastic Net": elastic_cv.best_params_
}
print("Mejores hiperparámetros:")
print(best_params)

# Obtener los mejores modelos
best_ridge = ridge_cv.best_estimator_
best_lasso = lasso_cv.best_estimator_
best_elastic = elastic_cv.best_estimator_

Mejores hiperparámetros:
{'Ridge': {'alpha': 27.825594022071257}, 'Lasso': {'alpha': 2.1544346900318843}, 'Elastic Net': {'alpha': 2.1544346900318843, 'l1_ratio': 0.9}}


In [ ]:
MODELS_NAME = ["Regresión Lineal", "Ridge (L2)", "Lasso (L1)", "Elastic Net"]

# Generar predicciones con los mejores modelos
y_pred_best_ridge = best_ridge.predict(X_test)
y_pred_best_lasso = best_lasso.predict(X_test)
y_pred_best_elastic = best_elastic.predict(X_test)

# Evaluar los modelos optimizados
mse_best_ridge = mean_squared_error(y_test, y_pred_best_ridge)
mse_best_lasso = mean_squared_error(y_test, y_pred_best_lasso)
mse_best_elastic = mean_squared_error(y_test, y_pred_best_elastic)

# Obtener el mejor modelo en términos de error cuadrático medio (MSE)
mse_models = [mse_lr, mse_best_ridge, mse_best_lasso, mse_best_elastic]
best_model_index = np.argmin(mse_models)
best_model_name = MODELS_NAME[best_model_index]
best_mse = mse_models[best_model_index]

# Mostrar el mejor modelo
echo = f"El mejor modelo es {best_model_name} con un MSE de {best_mse:.2f}"
print(echo)

# Comparar los modelos
mse_comparison = pd.DataFrame({
    "Modelo": MODELS_NAME,
    "MSE": [mse_lr, mse_best_ridge, mse_best_lasso, mse_best_elastic]
})

mse_comparison

El mejor modelo es Lasso (L1) con un MSE de 120.41


,Modelo,MSE
0,Regresión Lineal,128.930589
1,Ridge (L2),127.469233
2,Lasso (L1),120.409488
3,Elastic Net,120.417942


In [ ]:
# Mostrar ecuaciones resultantes sin monomios con coeficientes cero

def format_equation(intercept, coefs):
    terms = [f"{coefs[i]:.2f}*X{i+1}" for i in range(len(coefs)) if coefs[i] != 0]
    return f"Y = {' + '.join(terms)} + {intercept:.2f}" if terms else f"Y = {intercept:.2f}"

print("Ecuaciones resultantes:\n")
for model_name, model in zip(MODELS_NAME, [lin_reg, best_ridge, best_lasso, best_elastic]):
    equation = format_equation(model.intercept_, model.coef_)
    print(f"{model_name}: {equation}")

Ecuaciones resultantes:

Regresión Lineal: Y = -0.05*X1 + -0.14*X2 + 4.02*X3 + -0.07*X4 + 0.11*X5 + -2.39*X6 + -0.08*X7 + 2.85*X8 + -0.25*X9 + 0.04*X10 + -0.03*X11 + 0.15*X12 + -0.18*X13 + 0.43*X14 + -0.18*X15 + -0.63*X16 + -0.65*X17 + -0.32*X18 + -0.23*X19 + 0.02*X20 + 11.09
Ridge (L2): Y = -0.05*X1 + -0.14*X2 + 3.92*X3 + -0.06*X4 + 0.11*X5 + -2.34*X6 + -0.07*X7 + 2.78*X8 + -0.25*X9 + 0.04*X10 + -0.04*X11 + 0.15*X12 + -0.18*X13 + 0.42*X14 + -0.16*X15 + -0.61*X16 + -0.65*X17 + -0.32*X18 + -0.23*X19 + 0.02*X20 + 11.42
Lasso (L1): Y = 3.68*X3 + -2.14*X6 + 2.70*X8 + 0.15*X14 + -0.36*X16 + -0.44*X17 + -0.09*X18 + -0.00*X19 + 5.60
Elastic Net: Y = 3.60*X3 + -2.11*X6 + 2.65*X8 + -0.02*X9 + 0.17*X14 + -0.37*X16 + -0.46*X17 + -0.11*X18 + -0.03*X19 + 6.49


# ¿Cuándo la regresión lineal es mejor sin regularización?

- Cuando el número de muestras (n) es mucho mayor que el número de características (m).
    - Si n≫m, el modelo tiene suficiente información para aprender sin sobreajustar.
    - No hay riesgo significativo de sobreajuste.
    - Regularizar podría hacer que los coeficientes se reduzcan innecesariamente y perder información útil.
- Cuando no hay colinealidad entre las variables predictoras.
    - Si las variables son independientes, la regresión lineal estándar encuentra la mejor solución sin necesidad de ajustar coeficientes.
    - Ridge y Elastic Net ayudan cuando las variables están correlacionadas, pero si no lo están, no se necesita penalización.
- Cuando hay pocas variables irrelevantes o ruido bajo.
    - Si todas las variables son importantes y no hay muchas características irrelevantes, la regularización no es necesaria.
    - Lasso eliminaría variables importantes innecesariamente.
    - Ridge reduciría coeficientes útiles, haciendo el modelo menos interpretable.
- Cuando se necesita interpretar los coeficientes tal como son.
    - La regularización modifica los coeficientes, lo que puede dificultar la interpretación. Ejemplo:
        - Modelos en economía, medicina o ciencias sociales, donde cada coeficiente tiene un significado real (ej., cuánto influye el consumo de azúcar en el peso de una persona).
    - Ridge/Lasso pueden hacer que los coeficientes sean más difíciles de interpretar.

## Resumen

| Caso | ¿Se recomienda regularización? | Motivo |
|------|------------------------------|--------|
| n >> m **(muchas muestras, pocas variables)** | ❌ No | El modelo generaliza bien sin regularización. |
| **Variables independientes (sin colinealidad)** | ❌ No | No hay problemas de coeficientes inflados. |
| **Pocas variables irrelevantes o ruido bajo** | ❌ No | Regularizar podría eliminar o reducir coeficientes útiles. |
| **Necesidad de interpretar coeficientes exactos** | ❌ No | Regularización puede alterar la interpretación. |
